In [1]:
import foyer
import hoomd
from hoomd.data import make_snapshot
import mbuild as mb
from mbuild.formats.hoomd_simulation import create_hoomd_simulation
import numpy as np

from planckton.utils.rigid import connect_rings

/Users/jenny/miniconda3/envs/planckton/lib/python3.7/site-packages/mdtraj/core/trajectory.py:32: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [2]:
bz_str = "c1ccccc1"
bz = mb.load(bz_str, smiles=True, ignore_box_warn=True)
name = "bz"
bz.name = name
#bz.visualize().show()

In [3]:
#npt_str = "c1ccc2ccccc2c1"
#npt = mb.load(npt_str, smiles=True, ignore_box_warn=True)
#npt.visualize().show()

In [4]:
#py_str = "c1cc2cccc3ccc4cccc1c4c32"
#py = mb.load(py_str, smiles=True, ignore_box_warn=True)
#py.visualize().show()

In [5]:
box = mb.Box([5,5,5])
system = mb.fill_box(bz, n_compounds=10, box=box)
#system.visualize().show()

In [6]:
gaff = foyer.forcefields.load_GAFF()
pmd_system = system.to_parmed(residues=[name])
typed_system = gaff.apply(system)
#print(set([atom.type for atom in typed_system.atoms]))

/Users/jenny/miniconda3/envs/planckton/lib/python3.7/site-packages/foyer/forcefield.py:449: UserWarning: No force field version number found in force field XML file.
  'No force field version number found in force field XML file.'
/Users/jenny/miniconda3/envs/planckton/lib/python3.7/site-packages/foyer/forcefield.py:461: UserWarning: No force field name found in force field XML file.
  'No force field name found in force field XML file.'
/Users/jenny/miniconda3/envs/planckton/lib/python3.7/site-packages/foyer/validator.py:132: ValidationWarning: You have empty smart definition(s)
  warn("You have empty smart definition(s)", ValidationWarning)


On feat/rigid branch of my fork of mbuild, an initial snapshot can be passed in:

```
create_hoomd_simulation(structure, ref_distance=1.0, ref_mass=1.0, ref_energy=1.0, r_cut=1.2, auto_scale=False, snapshot_kwargs={}, pppm_kwargs={'Nx': 8, 'Ny': 8, 'Nz': 8, 'order': 4}, init_snap=None)
```

First, make a snapshot with 10 rigid particles--one for each benzene ring:

In [7]:
sim = hoomd.context.SimulationContext()

with sim:
    hoomd.context.initialize("")
    init_snap = make_snapshot(N=10, particle_types=["_R"], box=hoomd.data.boxdim(L=10))

HOOMD-blue 2.9.3 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 10/17/2020
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, J Glaser, and S C Glotzer. "HOOMD-blue: A Python package for
  high-performance molecular dynamics and hard particle Monte Carlo
  simulations", Computational Materials Science 173 (2020) 109363
-----
HOOMD-blue is running on the CPU


In [8]:
with sim:
    hoomd_objects, ref_values = create_hoomd_simulation(
        typed_system, auto_scale=True, init_snap=init_snap
    )
    snap = hoomd_objects[0]

notice(2): Group "all" created containing 130 particles
notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 10
notice(2): Particles with 3 exclusions             : 60
notice(2): Particles with 7 exclusions             : 60
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
Processing LJ and QQ
notice(2): Group "charged" created containing 0 particles
No charged groups found, ignoring electrostatics
Processing 1-4 interactions, adjusting neighborlist exclusions
Processing harmonic bonds
Processing harmonic angles
Processing periodic torsions
HOOMD SimulationContext updated from ParmEd Structure


Want to do something with [ring detection](https://openbabel.org/wiki/Ring_detection) where rings are automatically converted to rigid bodies.

- convert to pybel mol
- use smarts matching to find rings
- make rigid

[SSSR documentation](http://openbabel.org/dev-api/classOpenBabel_1_1OBRing.shtml#_details)

In [9]:
system_mol = system.to_pybel()
rings = sorted(connect_rings(system_mol), key=lambda x: x[0])

print(*rings, sep="\n")
print(len(rings))

[0 1 2 3 4 5]
[12 13 14 15 16 17]
[24 25 26 27 28 29]
[36 37 38 39 40 41]
[48 49 50 51 52 53]
[64 65 60 61 62 63]
[72 73 74 75 76 77]
[84 85 86 87 88 89]
[ 96  97  98  99 100 101]
[108 109 110 111 112 113]
10


In [10]:
for i,ring in enumerate(rings):
    ring += len(rings)
    snap.particles.position[i] = np.mean(snap.particles.position[ring], axis=0)
    snap.particles.body[i] = i
    snap.particles.body[ring] = i * np.ones(len(ring))

In [14]:
snap.particles.body

array([         0,          1,          2,          3,          4,
                5,          6,          7,          8,          9,
                0,          0,          0,          0,          0,
                0, 4294967295, 4294967295, 4294967295, 4294967295,
       4294967295, 4294967295,          1,          1,          1,
                1,          1,          1, 4294967295, 4294967295,
       4294967295, 4294967295, 4294967295, 4294967295,          2,
                2,          2,          2,          2,          2,
       4294967295, 4294967295, 4294967295, 4294967295, 4294967295,
       4294967295,          3,          3,          3,          3,
                3,          3, 4294967295, 4294967295, 4294967295,
       4294967295, 4294967295, 4294967295,          4,          4,
                4,          4,          4,          4, 4294967295,
       4294967295, 4294967295, 4294967295, 4294967295, 4294967295,
                5,          5,          5,          5,        

In [13]:
snap.particles.position

array([[-1.4753485 , -2.2619445 , -3.6145627 ],
       [-2.8375711 , -5.605292  , -3.0110047 ],
       [-3.143246  , -2.9563377 ,  4.407785  ],
       [-5.3500056 , -1.5039836 ,  2.6052277 ],
       [-5.9524727 ,  2.1669724 ,  3.4030674 ],
       [ 5.3233447 , -2.262244  ,  5.5795    ],
       [-0.1939019 ,  4.9484797 ,  5.3657055 ],
       [-3.394964  ,  5.445753  ,  4.172228  ],
       [-6.62466   , -3.6519563 , -2.780978  ],
       [ 4.0822525 , -1.4754806 ,  1.6986548 ],
       [-1.5133265 , -1.9395232 , -3.8653228 ],
       [-1.3491597 , -2.2891803 , -4.004143  ],
       [-1.3165853 , -2.6132596 , -3.7545214 ],
       [-1.4368455 , -2.5841992 , -3.3636713 ],
       [-1.597495  , -2.2333288 , -3.223781  ],
       [-1.638679  , -1.9121761 , -3.4759364 ],
       [-1.5440898 , -1.688826  , -4.0613537 ],
       [-1.2471571 , -2.3090792 , -4.306221  ],
       [-1.197387  , -2.8885994 , -3.8651645 ],
       [-1.4053987 , -2.834745  , -3.167557  ],
       [-1.6915462 , -2.210766  , -2.919

In [60]:
sim = hoomd.context.SimulationContext()

with sim:
    hoomd.init.read_snapshot(snap)
    for i in hoomd_objects[1:]:
        i
    rigid = hoomd.md.constrain.rigid()
    r_pos = snap.particles.position[ring[0]]
    const_pos = snap.particles.position[ring[1:]]
    const_pos -= r_pos
    rigid.set_param(
        "_R", 
        types=[snap.particles.types[i] for i in snap.particles.typeid[ring[1:]]],
        positions=[tuple(i) for i in const_pos]
    )
    rigid.validate_bodies()

    centers = hoomd.group.rigid()
    nonrigid = hoomd.group.nonrigid()

    hoomd.md.integrate.langevin(group=centers, kT=1.0, seed=42);
    hoomd.md.integrate.langevin(group=nonrigid, kT=1.0, seed=42);
    hoomd.run(1e4)

notice(2): Group "all" created containing 120 particles


**ERROR**: constrain.rigid(): Central particles must have a body tag identical to their contiguous tag.


RuntimeError: Error validating rigid bodies


In [16]:
help(rigid.validate_bodies)

Help on method validate_bodies in module hoomd.md.constrain:

validate_bodies() method of hoomd.md.constrain.rigid instance
    Validate that bodies are well defined and prepare for the simulation run.

